# Sentiment Analysis of Climate Change Tweets

[Insert intro and explanation to the project]

## Data Retrieval
For this project I am going to use two sources of data. 

The first one is a dataset from [kaggle.com](https://www.kaggle.com/datasets/edqian/twitter-climate-change-sentiment-dataset) containing information about 43943 tweets shared in between Apr 27, 2015 and Feb 21, 2018. For each tweet the full text of the tweet and the sentiment of the tweet is reported (more on this below). I plan on using this dataset to train a machine learning algorithm that classifies the tweets' sentiment out of sample with high accuracy. Unfortunately, I did not manage to get in touch with the creator of this dataset to get information on which restrictions he used to extract the tweets. This information would be relevant for me when extracting additional tweets for further analysis.

The second source of data I am going to use comes from Twitter. I use the Twitter's API to extract new tweets following criteria similar to those used for building the training dataset ([if I manage to find the guy; otherwise: arbitrary; maybe I'll try to see if he uses a specific collection of hashtags]). I plan on using my model to classify these tweets. (further info on this when you actually extract it; it would be interesting to tkae tweets after 2018)

### Training Dataset
Here I load the training dataset from Kaggle and take a peek at what it contains.

## Data Arrangement and Checks

## Pre-processing

## Final Data

## Algorithm

## Visualization

## Potential Analysis and Future Uses